In [ ]:
# Cell 1: Imports
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display

# Cell 2: Charger les logs
logs = []
with open("logs/api.log", "r", encoding="utf-8") as f:
    for line in f:
        try:
            logs.append(json.loads(line.strip()))
        except json.JSONDecodeError:
            continue

df = pd.DataFrame(logs)
df["timestamp"] = pd.to_datetime(df["timestamp"])

print(f"Total logs: {len(df)}")
print(f"Période: {df['timestamp'].min()} to {df['timestamp'].max()}")

# Cell 3: Requêtes par heure
df_hourly = df.set_index("timestamp").resample("1H").size()

plt.figure(figsize=(12, 4))
df_hourly.plot(kind="line", marker="o")
plt.title("Requêtes par heure")
plt.ylabel("Nombre de requêtes")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Cell 4: Distribution des temps de réponse
plt.figure(figsize=(10, 4))
plt.hist(df["duration_ms"], bins=50, edgecolor="black", alpha=0.7)
plt.axvline(df["duration_ms"].quantile(0.95), color="red", 
            linestyle="--", label="P95")
plt.axvline(df["duration_ms"].quantile(0.50), color="green",
            linestyle="--", label="P50")
plt.title("Distribution des temps de réponse")
plt.xlabel("Durée (ms)")
plt.ylabel("Fréquence")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Cell 5: Taux d'erreur par endpoint
df["is_error"] = df["status_code"] >= 400
error_by_endpoint = df.groupby("path")["is_error"].mean() * 100

plt.figure(figsize=(10, 4))
error_by_endpoint.sort_values().plot(kind="bar")
plt.title("Taux d'erreur par endpoint (%)")
plt.ylabel("Taux d'erreur %")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Cell 6: Top 10 endpoints
top_paths = df["path"].value_counts().head(10)
print("Top 10 endpoints:")
print(top_paths)

# Cell 7: Analyse des codes de statut HTTP
status_counts = df["status_code"].value_counts()
print("\nDistribution des codes HTTP:")
print(status_counts)

# Cell 8: Méthodes HTTP utilisées
method_counts = df["method"].value_counts()
print("\nMéthodes HTTP:")
print(method_counts)

# Cell 9: Dashboard résumé
print("\n" + "="*50)
print("DASHBOARD DE MONITORING")
print("="*50)
print(f"Période d'analyse: {df['timestamp'].min()} à {df['timestamp'].max()}")
print(f"Total requêtes: {len(df):,}")
print(f"Taux d'erreur global: {df['is_error'].mean()*100:.2f}%")
print(f"Latence moyenne: {df['duration_ms'].mean():.2f} ms")
print(f"Latence P95: {df['duration_ms'].quantile(0.95):.2f} ms")
print(f"Latence P99: {df['duration_ms'].quantile(0.99):.2f} ms")
print(f"Endpoints uniques: {df['path'].nunique()}")
print(f"IPs clientes uniques: {df['client_ip'].nunique()}")